In [12]:
!pip install -qqq huggingface-hub python-dotenv transformers sentence-transformers langchain-community langchain-huggingface langchain tqdm regex gradio unidecode pymupdf chromadb bitsandbytes langchain_chroma faiss-gpu --progress-bar off

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 26.7 MB/s eta 0:00:00


In [13]:
#!pip install -U bitsandbytes --progress-bar off
!pip install -qqq huggingface-hub python-dotenv transformers sentence-transformers langchain-community langchain-huggingface langchain tqdm regex gradio unidecode pymupdf chromadb bitsandbytes --progress-bar off

In [ ]:
# # questo serve
# import requests
# import zipfile
# import os

# # URL del file ZIP
# url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/hotpotqa.zip"

# # Nome del file ZIP locale
# zip_file_name = "hotpotqa.zip"

# # Cartella di estrazione
# extract_dir = "hotpotqa"

# # Scaricare il file ZIP
# print("Scaricamento del file ZIP...")
# response = requests.get(url, stream=True)
# if response.status_code == 200:
#     with open(zip_file_name, "wb") as f:
#         for chunk in response.iter_content(chunk_size=1024):
#             f.write(chunk)
#     print(f"Scaricamento completato: {zip_file_name}")
# else:
#     print(f"Errore durante il download: {response.status_code}")
#     exit()

# # Estrarre il contenuto del file ZIP
# print(f"Estrazione del file ZIP in {extract_dir}...")
# os.makedirs(extract_dir, exist_ok=True)
# with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
#     zip_ref.extractall(extract_dir)
# print(f"Estrazione completata. Contenuti disponibili nella cartella: {extract_dir}")

# # remove zip file after extraction
# os.remove(zip_file_name)
# print(f"File ZIP rimosso: {zip_file_name}")


Scaricamento del file ZIP...
Scaricamento completato: hotpotqa.zip
Estrazione del file ZIP in hotpotqa...
Estrazione completata. Contenuti disponibili nella cartella: hotpotqa
File ZIP rimosso: hotpotqa.zip


In [14]:
import json

def print_first_three_elements(file_path):
    """
    Legge un file JSON Lines (.jsonl) e stampa i primi tre elementi.

    :param file_path: Percorso al file corpus.jsonl
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            count = 0
            for line in file:
                # Decodifica ogni riga come oggetto JSON
                data = json.loads(line.strip())
                print(data)
                count += 1
                # Ferma dopo aver letto 3 elementi
                if count == 3:
                    break
    except FileNotFoundError:
        print(f"File non trovato: {file_path}")
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing del file JSONL: {e}")

# Esempio di utilizzo
file_path = "hotpotqa/hotpotqa/corpus.jsonl"
print_first_three_elements(file_path)
# Calcola e stampa il numero totale di documenti
with open(file_path, 'r', encoding='utf-8') as file:
    total_documents = sum(1 for _ in file)
print(f"Numero totale di documenti: {total_documents}")

{'_id': '12', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy that advocates self-governed societies based on voluntary institutions. These are often described as stateless societies, although several authors have defined them more specifically as institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary and harmful.', 'metadata': {'url': 'https://en.wikipedia.org/wiki?curid=12'}}
{'_id': '25', 'title': 'Autism', 'text': "Autism is a neurodevelopmental disorder characterized by impaired social interaction, impaired verbal and non-verbal communication, and restricted and repetitive behavior. Parents usually notice signs in the first two years of their child's life. These signs often develop gradually, though some children with autism reach their developmental milestones at a normal pace and then regress. The diagnostic criteria require that symptoms become apparent in early childhood, typically before age three.

In [15]:
from dotenv import load_dotenv
import os
from huggingface_hub import login


try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("Running in Google Colab. Using userdata to get HF_TOKEN.")
except ModuleNotFoundError:
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')
    print("Not running in Google Colab. Using load_dotenv to get HF_TOKEN.")

login(token=HF_TOKEN)

Running in Google Colab. Using userdata to get HF_TOKEN.


In [16]:
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

if torch.backends.mps.is_available():
    device = "mps"
    print("MPS is available. Working on MPS.")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("CUDA is available. Working on GPU.")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("CUDA and MPS not available. Working on CPU.")

CUDA is available. Working on GPU.
GPU name: NVIDIA A100-SXM4-40GB


In [19]:
import json
import re
from unidecode import unidecode
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents.base import Document

# Carica documenti dal file JSON Lines
def load_documents(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line.strip())
            if "text" in data:
                documents.append(Document(page_content=data["text"], metadata={"title": data.get("title", "")}))
    total_length = sum(len(doc.page_content) for doc in documents)
    avg_length = total_length / len(documents) if documents else 0
    print(f"Average length of documents: {avg_length:.3f}")
    return documents

# Preprocessa i testi
def preprocess_text(text):
    # Rimuovi formule matematiche
    text = re.sub(r'\$.*?\$', '', text)
    # Rimuovi URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Rimuovi tag HTML
    text = re.sub(r'<.*?>', '', text)
    # Rimuovi caratteri non ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Rimuovi caratteri speciali e numeri, mantenendo solo lettere e punteggiatura di base
    text = re.sub(r'[^a-zA-Z\s.,;:!?\'"-]', '', text)
    # Normalizza Unicode
    text = unidecode(text)
    # Rimuovi spazi multipli
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Percorso al file del dataset
documents = load_documents(file_path)

# Preprocessa i contenuti
for doc in tqdm(documents, desc="Preprocessing documents"):
  doc.page_content = preprocess_text(doc.page_content)

# TODO : i documenti sono già corti, non serve splittarli
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# split_documents = []
# for doc in tqdm(documents, desc="Preprocessing documents"):
#     if len(doc["page_content"]) > 1500:
#         chunks = text_splitter.split_text(doc["page_content"])
#         split_documents.extend([{"page_content": chunk, "metadata": doc["metadata"]} for chunk in chunks])

Average length of documents: 267.998


Preprocessing documents: 100%|██████████| 5233329/5233329 [03:09<00:00, 27636.95it/s]


In [20]:
# prompt: delete the local folder "embeddings" and all it contains

import shutil
import os

# Delete the "embeddings" folder and its contents
folder_path = "embeddings"
if os.path.exists(folder_path):
    try:
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' and its contents have been deleted successfully.")
    except OSError as e:
        print(f"Error deleting folder '{folder_path}': {e}")
else:
    print(f"Folder '{folder_path}' does not exist.")

Folder 'embeddings' does not exist.


In [25]:
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
import faiss
import os
from tqdm import tqdm

# Directory to save embeddings and FAISS index
output_dir = "embeddings"
faiss_dir = "faiss_index"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(faiss_dir, exist_ok=True)

# Configure the model for GPU usage
model_kwargs = {'device': 'cuda'}  # Ensure GPU acceleration
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs=model_kwargs)

def save_embeddings(documents, batch_size=64, output_dir="embeddings", prefix_name="embeddings"):
    all_embeddings_paths = []
    for i in tqdm(range(0, len(documents), batch_size), desc="Generating embeddings"):
        batch_docs = documents[i:i+batch_size]
        batch_texts = [doc.page_content for doc in batch_docs]  # Extract text content
        batch_embeddings = embeddings_model.embed_documents(batch_texts)
        batch_embeddings_path = os.path.join(output_dir, f"{prefix_name}_{i}_embeddings.npy")
        np.save(batch_embeddings_path, batch_embeddings)
        all_embeddings_paths.append(batch_embeddings_path)

    return all_embeddings_paths

def load_and_concatenate_embeddings(embeddings_paths):
    all_embeddings = []
    for path in embeddings_paths:
        batch_embeddings = np.load(path)
        all_embeddings.append(batch_embeddings)
    all_embeddings = np.concatenate(all_embeddings, axis=0)

    all_embeddings = np.ascontiguousarray(all_embeddings, dtype=np.float32)

    return all_embeddings

def index_embeddings(embeddings, faiss_dir="faiss_index", index_file_name="index.faiss"):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, os.path.join(faiss_dir, index_file_name))
    print(f"FAISS index saved at {os.path.join(faiss_dir, index_file_name)}")

# Save embeddings
embeddings_paths = save_embeddings(documents, batch_size=1024, output_dir=output_dir, prefix_name="embeddings")

# Load and concatenate embeddings
embeddings = load_and_concatenate_embeddings(embeddings_paths)

# Index embeddings using FAISS
index_embeddings(embeddings, faiss_dir=faiss_dir, index_file_name="index.faiss")

# Create retriever (example usage, not implemented)
#retriever = FAISSRetriever(index_path=os.path.join(faiss_dir, "index.faiss"))

FAISS index saved at faiss_index/index.faiss


In [24]:
import os
import numpy as np
from tqdm import tqdm

def load_embeddings_from_folder(embeddings_dir, prefix_name="embeddings"):
    """
    Carica gli embeddings già salvati in una cartella.

    :param embeddings_dir: La cartella contenente i file di embeddings.
    :param prefix_name: Il prefisso dei nomi dei file (default: "embeddings").
    :return: Un array numpy con tutti gli embeddings concatenati.
    """
    all_embeddings = []
    all_embeddings_paths = []

    # Ottieni tutti i file con il prefisso specificato
    for filename in tqdm(os.listdir(embeddings_dir), desc="Loading embeddings"):
        if filename.startswith(prefix_name) and filename.endswith(".npy"):
            file_path = os.path.join(embeddings_dir, filename)
            all_embeddings_paths.append(file_path)
            embeddings = np.load(file_path)
            all_embeddings.append(embeddings)

    # Concatena tutti gli embeddings in un'unica matrice
    all_embeddings = np.concatenate(all_embeddings, axis=0)

    all_embeddings = np.ascontiguousarray(all_embeddings, dtype=np.float32)


    print(f"Loaded {len(all_embeddings)} embeddings from {len(all_embeddings_paths)} files.")

    return all_embeddings, all_embeddings_paths

# Esempio di utilizzo
embeddings_dir = "embeddings"  # Cartella che contiene gli embeddings salvati
embeddings, embeddings_paths = load_embeddings_from_folder(embeddings_dir)


Loading embeddings: 100%|██████████| 5111/5111 [00:10<00:00, 467.30it/s]


Loaded 5233329 embeddings from 5111 files.


In [51]:
import faiss
import numpy as np
from typing import List, Tuple
from langchain_huggingface import HuggingFaceEmbeddings

class FaissRetriever:
    """
    A retriever that uses a FAISS index to perform nearest neighbor search on precomputed document embeddings.
    """
    def __init__(self,
                 index_path: str,
                 embeddings_model_name: str,
                 index_id_to_db_id: dict = None,
                 device: str = "cuda"):
        """
        Initialize the retriever.

        :param index_path: Path to the FAISS index file.
        :param embeddings_model_name: Name of the embeddings model to use for encoding queries.
        :param index_id_to_db_id: Optional mapping from index IDs to database IDs.
        :param device: Device to run the embedding model on ("cuda", "cpu").
        """
        self.index = faiss.read_index(index_path)
        self.embeddings_model = HuggingFaceEmbeddings(
            model_name=embeddings_model_name,
            model_kwargs={'device': device}
        )
        # If no mapping is provided, create a default mapping
        self.index_id_to_db_id = index_id_to_db_id or {i: str(i) for i in range(self.index.ntotal)}

    def encode_query(self, query: str) -> np.ndarray:
        """
        Encode a query string into an embedding vector.

        :param query: The query string to encode.
        :return: A numpy array representing the query embedding.
        """
        query_embedding = self.embeddings_model.embed_query(query)
        # Convert query_embedding to a contiguous numpy array of float32
        return np.array(query_embedding, dtype=np.float32).reshape(1, -1)

    def retrieve(self, query: str, top_k: int = 5):
        """
        Retrieve the top-k nearest neighbors for a given query.

        :param query: The query string to search for.
        :param top_k: Number of top results to return.
        :return: A list of (index, distance) tuples for the top-k results.
        """
        query_embedding = self.encode_query(query)
        distances, indices = self.index.search(query_embedding, top_k)
        return list(zip(indices[0], distances[0]))

    def search_knn(
      self,
      query_vectors: np.ndarray,
      top_docs: int,
      index_batch_size: int = 2048
      ) -> List[Tuple[List[str], List[float]]]:
      """
      Performs a k-nearest neighbor search for the given query vector(s).

      Args:
          query_vectors (np.ndarray): A numpy array of query vector(s).
          top_docs (int): The number of top documents to return for each query.
          index_batch_size (int): The batch size to use when indexing.

      Returns:
          A list of tuples, each containing a list of document IDs and a list of corresponding scores.
      """
      # Ensure query_vectors is a 2D array, even for a single query
      if query_vectors.ndim == 1:
          query_vectors = query_vectors.reshape(1, -1)

      result = []
      # Calculate number of batches
      nbatch = (len(query_vectors) - 1) // index_batch_size + 1

      for k in range(nbatch):
          # Calculate start and end indices for current batch
          start_idx = k * index_batch_size
          end_idx = min((k+1) * index_batch_size, len(query_vectors))

          # Extract current batch of query vectors
          q = query_vectors[start_idx:end_idx]

          # Perform similarity search
          scores, indexes = self.index.search(q, top_docs)

          # Convert index IDs to database IDs
          db_ids = [
              [str(self.index_id_to_db_id.get(i, i)) for i in query_top_idxs]
              for query_top_idxs in indexes
          ]

          # Extend result with current batch results
          result.extend([(db_ids[i], scores[i]) for i in range(len(db_ids))])

      return result

In [52]:
retriever = FaissRetriever(
    index_path="faiss_index/index.faiss",
    embeddings_model_name="sentence-transformers/all-MiniLM-L6-v2",
    device="cuda"  # Usa "cpu" se non hai GPU
)

In [57]:
query = "What country of origin does House of Cosbys and Bill Cosby have in common?"
# Encode the query to get the query vector
single_query_vector = retriever.encode_query(query)

# Perform KNN search
single_result = retriever.search_knn(single_query_vector, top_docs=100)

print("Top-k results:")
for db_ids, distances in single_result:
    for doc_id, distance in zip(db_ids, distances):
        print(f"Document ID: {doc_id}, Distance: {distance}")

Top-k results:
Document ID: 1897486, Distance: 0.7028908729553223
Document ID: 4092194, Distance: 0.7716771364212036
Document ID: 130815, Distance: 0.8594081401824951
Document ID: 1906561, Distance: 0.9394850134849548
Document ID: 3701716, Distance: 0.9780150651931763
Document ID: 128482, Distance: 0.9825448393821716
Document ID: 3983874, Distance: 0.9914695024490356
Document ID: 133838, Distance: 1.0045663118362427
Document ID: 669269, Distance: 1.0084073543548584
Document ID: 2690175, Distance: 1.0117554664611816
Document ID: 688474, Distance: 1.0125784873962402
Document ID: 1897508, Distance: 1.0241860151290894
Document ID: 2341247, Distance: 1.0268267393112183
Document ID: 3044080, Distance: 1.0327492952346802
Document ID: 3886838, Distance: 1.042952537536621
Document ID: 4540257, Distance: 1.0436501502990723
Document ID: 1272588, Distance: 1.049180507659912
Document ID: 3300222, Distance: 1.0573095083236694
Document ID: 4598642, Distance: 1.0573601722717285
Document ID: 520747, Di

In [58]:
for db_ids, distances in single_result:
    for doc_id, distance in zip(db_ids, distances):
        print(f"Document ID: {doc_id}")
        print("Document Content:", documents[int(doc_id)].page_content)
        print(f"Distance: {distance}")
        print("-" * 50)  # Separator between documents

Document ID: 1897486
Document Content: KVOW AM is a radio station broadcasting a TalkPersonality format. Licensed to Riverton, Wyoming, United States, the station is currently owned by Edwards Communications, Lc.
Distance: 0.7028908729553223
--------------------------------------------------
Document ID: 4092194
Document Content: Katha nankunshanica is a moth of the family Erebidae. It is found in China Guangdong.
Distance: 0.7716771364212036
--------------------------------------------------
Document ID: 130815
Document Content: Alexander Campbell September March was a Scots-Irish immigrant who became an ordained minister in the United States and joined his father Thomas Campbell as a leader of a reform effort that is historically known as the Restoration Movement, and by some as the "Stone-Campbell Movement." It resulted in the development of non-denominational Christian churches, which stressed reliance on scripture and few essentials.
Distance: 0.8594081401824951
------------------

In [ ]:
from pathlib import Path

def get_folder_size(folder_path):
    folder = Path(folder_path)
    return sum(f.stat().st_size for f in folder.rglob('*') if f.is_file())

# Percorso della cartella
folder_path = "./embeddings"

# Calcola la dimensione in byte
folder_size = get_folder_size(folder_path)

# Converti in formato leggibile
print(f"Size of '{folder_path}': {folder_size / (1024 ** 2):.2f} MB")


Size of './embeddings': 15332.64 MB


In [ ]:
# prompt: download the local folder faiss_index

from google.colab import files
import shutil

# Define the source and destination paths
source_folder = "faiss_index"
destination_zip = "faiss_index.zip"


# Create a zip archive of the folder
shutil.make_archive(destination_zip.replace(".zip", ""), "zip", source_folder)

# now the zip can be downloaded

In [ ]:
# from tqdm import tqdm
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma

# # Configurazione modello
# model_kwargs = {'device': 'cuda'}
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs=model_kwargs)

# batch_size = 512

# # Creazione del database Chroma con aggiunta batch
# print("Creating Chroma database...")
# db = Chroma(
#     persist_directory="chroma_db",
#     embedding_function=embeddings
# )

# # Dividi i documenti in batch e usa tqdm per mostrare la progress bar
# for i in tqdm(range(0, len(documents), batch_size), desc="Indexing documents in batches", unit="batch"):
#     batch = documents[i:i + batch_size]
#     db.add_documents(batch)  # Aggiunge i documenti batch per batch

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.repetition_penalty = 1.6  # Add repetition penalty



model.to(device)

In [ ]:
from sentence_transformers import SentenceTransformer


# Initialize the sentence-transformers model
embedder_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder_model = SentenceTransformer(embedder_model_name)

def embedder(chunk):
    embeddings = embedder_model.encode(chunk, convert_to_tensor=True)
    return embeddings.cpu().numpy()


# Use the retriever to get the top_k_chunks for a given query
def search(query, retriever, k=10):
    results = retriever.invoke(query, k=k)
    return [result.page_content for result in results]

## PROMPT AND ANSWER QUESTION FUNCTION

In [ ]:
base_prompt = """You are an AI assistant for RAG. Your task is to understand the user question, and provide an answer using the provided contexts.

Your answers are correct, high-quality, and written by a domain expert. If the provided context does not contain the answer, simply state, "The provided context does not have the answer."

User question: {user_query}

Contexts:
{chunks_information}

Answer:
"""

In [ ]:
import time

def answer_questions(questions):
    count = 0
    error = 0
    results = {}

    current_time = time.strftime("%m%d-%H%M%S")

    pbar = tqdm(questions, total=len(questions), desc="Answering questions...", unit="question")
    for q in pbar:
        top_k_chunks = search(q['question'], retriever, k=10)
        retrieved_chunks = [chunk for chunk in top_k_chunks]
        prompt = base_prompt.format(user_query=q['question'], chunks_information="\n".join(retrieved_chunks))
        encoding = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.inference_mode():
            outputs = model.generate(
                input_ids=encoding.input_ids,
                attention_mask=encoding.attention_mask,
                generation_config=generation_config,
                num_beams=5,  # Use beam search for better results
                early_stopping=True,  # Stop early if all beams finish
            )

        # Exclude the prompt tokens from the generated output
        generated_tokens = outputs[0][len(encoding.input_ids[0]):]
        generated_unpreprocessed_sequence = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        match = re.search(r'\b[1-4]\b', generated_unpreprocessed_sequence)
        answer = match.group(0) if match else ""  # first number found or empty string

        with open(f"quiz/runs_basemodel/quiz_answers_{current_time}.txt", "a", encoding="utf-8") as f:
            f.write(f"Question: {q['question']}\nAnswer: {answer}\nCorrect answer:{q['correct']}\nGenerated unpreprocessed sequence: {generated_unpreprocessed_sequence}\n--------------------------------------------------------------------\n\n")

        results[q['question_id']] = answer

        if len(answer) != 1 or answer not in "1234":
            error += 1
        else:  # the format is correct, now check if the answer is correct
            if str(q['correct']) == answer:
                count += 1
        pbar.set_postfix(Corrects=f"{count}/{len(questions)}", Errors=error)

    print("-------------------------\tFINISHED RUN. Error count: ", error, "-------------------------")
    return results, count / len(questions) * 100

# Example questions
# questions = [
#     {"question": "", "correct": "1", "question_id": "1"},
#     # Add more questions as needed
# ]

# results, score = answer_questions(questions)
# print(f"Final score: {score}%")

## GRADIO

In [ ]:
import gradio as gr

def query_rag_model(user_query):
    top_k_chunks = search(user_query, retriever, k=10)
    retrieved_chunks = [chunk for chunk in top_k_chunks]
    prompt = base_prompt.format(user_query=user_query, chunks_information="\n".join(retrieved_chunks))
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
            num_beams=5,
            early_stopping=True,
        )
    generated_tokens = outputs[0][len(encoding.input_ids[0]):]
    generated_unpreprocessed_sequence = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return generated_unpreprocessed_sequence

iface = gr.Interface(
    fn=query_rag_model,
    inputs="text",
    outputs="text",
    title="RAG Model Query Interface",
    description="Ask questions to the RAG model and get answers based on the provided PDF context."
)

iface.launch()